# Initialization

In [20]:
import os, sys
import time
import numpy as np
import pandas as pd
import random
from scipy import stats as st
import itertools
import operator

import torch

from tqdm.notebook import trange
from tqdm import tqdm

random_state = np.random.RandomState(2020)

In [21]:
# get currently working directory
base_dir = os.getcwd()

# load functions from other notebooks
helpers_file = os.path.join(base_dir, 'helpers.ipynb')
%run $helpers_file

# Load spotlight module
for p in ['../spotlight_ext']:
    module_path = os.path.abspath(os.path.join(base_dir, p))
    if module_path not in sys.path:
        sys.path.append(module_path)

# Load Dataset

## Models

In [22]:
lstm_model = load_model(model_type='entire')
pooling_model = load_model('pooling')

pretrained_models = {
    'lstm': lstm_model,
    'pooling': pooling_model,
}

## Dataset

In [23]:
from spotlight.cross_validation import random_train_test_split
from spotlight.datasets.movielens import get_movielens_dataset

# get dataset
dataset = get_movielens_dataset(variant='1M')
train, test = random_train_test_split(dataset, random_state=random_state)

max_sequence_length = 20
train = train.to_sequence(max_sequence_length=max_sequence_length)
test = test.to_sequence(max_sequence_length=max_sequence_length)

# Genetic Search

## Initialization

target item is 149 in this test case, top k is 10

In [24]:
test_interaction = test.sequences[test.user_ids == 3][0].copy()
test_interaction = test_interaction[test_interaction != 0]
test_interaction.sort()
test_interaction

array([ 59, 114, 124, 125, 177, 186, 190, 191, 196, 197, 200], dtype=int32)

In [25]:
len_test_interaction = len(test_interaction)
len_test_interaction

11

In [26]:
def get_position_item(model, test_interaction, position=1):
    prediction = model.predict(test_interaction)
    prediction[test_interaction] = -StaticVars.FLOAT_MAX
    rk_data = st.rankdata(-prediction, method='ordinal')
    index = np.where(rk_data == position)
    return index[0][0]

In [27]:
get_position_item(pooling_model, test_interaction, 2)

167

## Random CF candidate selection

In [28]:
import numpy as np

def generate_random_sublists(original_list, sublists_info):
    result_sublists = []
    rng = np.random.default_rng(seed=2020)  # Seed for reproducibility

    for length, count in sublists_info.items():
        generated_sublists_for_length = set()

        while len(generated_sublists_for_length) < count:
            sublist = tuple(rng.choice(original_list, length, replace=False))
            generated_sublists_for_length.add(sublist)

        result_sublists.extend(np.array(list(sublist)) for sublist in generated_sublists_for_length)

    return result_sublists


## Crossover and Mutation

In [29]:
import numpy as np

def crossover(first_list, second_list, p_cross):
    # Find the shorter length among the two lists
    length_first = len(first_list)
    length_second = len(second_list)
    shorter_length = min(length_first, length_second)
    
    # Compute the number of crossover points
    num_crossovers = int(shorter_length * p_cross)
    
    # Choose random indices for crossover within the range of shorter length
    rng = np.random.default_rng(seed=2020)
    crossover_indices_first = rng.choice(shorter_length, num_crossovers, replace=False)
    crossover_indices_second = rng.choice(shorter_length, num_crossovers, replace=False)
    
    # Sort the crossover indices
    crossover_indices_first.sort()
    crossover_indices_second.sort()
    
    # Swap the elements at the crossover indices
    for i in range(num_crossovers):
        index_first = crossover_indices_first[i]
        index_second = crossover_indices_second[i]
        first_list[index_first], second_list[index_second] = second_list[index_second], first_list[index_first]
    
    return first_list, second_list

def mutate_array(org_arr, arr_to_mutate, mutation_probability):
    # Calculate the number of elements to mutate
    num_mutations = int(mutation_probability * len(arr_to_mutate))
    rng = np.random.default_rng(seed=2020)
    # Select the indices to mutate
    indices_to_mutate = rng.choice(range(len(arr_to_mutate)), size=num_mutations, replace=False)
    
    # Mutate the selected elements
    for idx in indices_to_mutate:
        arr_to_mutate[idx] = rng.choice(org_arr)

    return arr_to_mutate


def remove_duplicates(arr):
    _, idx = np.unique(arr, return_index=True)
    return arr[np.sort(idx)]

## Loss Functions

In [30]:
from itertools import combinations, chain
import numpy as np
import scipy.stats as st

class StaticVars:
    FLOAT_MAX = float('inf')

def supersets_of_new_subsets_of_old(new_cf, old_cf):
    diff = np.setdiff1d(old_cf, new_cf)  # Elements that are in old_cf but not in new_cf
    for r in range(1, len(diff) + 1):
        for subset in combinations(diff, r):
            yield np.union1d(new_cf, subset)

def compute_yloss(target_score, kth_score):
    yloss = max(0, target_score / kth_score - 1.0)
    return yloss

def compute_distance(x, y):
    diff = np.setdiff1d(x, y)
    return len(diff)

def compute_loss(old_cf, new_cf, model, target_item, top_k, yloss_cache):
    cache_key = frozenset(new_cf)
    if cache_key in yloss_cache:
        yloss = yloss_cache[cache_key]
    else:
        new_prediction = model.predict(new_cf)
        new_prediction[new_cf] = -StaticVars.FLOAT_MAX
        new_rk_data = st.rankdata(-new_prediction, method='ordinal')

        top_k_index = np.where(new_rk_data == top_k)[0][0]
        yloss = compute_yloss(new_prediction[target_item], new_prediction[top_k_index])
        yloss_cache[cache_key] = yloss
    dis = compute_distance(old_cf, new_cf)

    subset_yloss = 0
    for superset in supersets_of_new_subsets_of_old(new_cf, old_cf):
        cache_key = frozenset(superset)
        if cache_key in yloss_cache:
            subset_yloss += yloss_cache[cache_key]
        else:
            subset_prediction = model.predict(superset)
            subset_prediction[superset] = -StaticVars.FLOAT_MAX
            sub_rk_data = st.rankdata(-subset_prediction, method='ordinal')
            sub_top_k_index = np.where(sub_rk_data == top_k)[0][0]
            subset_yloss += compute_yloss(subset_prediction[target_item], subset_prediction[sub_top_k_index])
            yloss_cache[cache_key] = subset_yloss

    return list([yloss, dis, subset_yloss])


# NSGA-II
Apply NSGA-II to the problem of finding the optimal candicates in multi-objective optimization problem.
Based on:
- Non-domination Rank
- Crowding Distance

In [31]:
def dominates(row, candidateRow):
    """Determine if one solution dominates another"""
    return all(r <= cr for r, cr in zip(row, candidateRow)) and any(r < cr for r, cr in zip(row, candidateRow))

def crowding_distance_assignment(front, values):
    distances = [0] * len(values)  # Initialize the distance for every solution as 0
    num_objs = len(values[0])
    
    for m in range(num_objs):
        sorted_front = sorted(front, key=lambda x: values[x][m])

        # Assign infinite distance at boundaries.
        distances[sorted_front[0]] = distances[sorted_front[-1]] = float('inf')

        # Normalize the objective values for distance computation.
        obj_min = values[sorted_front[0]][m]
        obj_max = values[sorted_front[-1]][m]
        denom = obj_max - obj_min if obj_max != obj_min else 1

        for i in range(1, len(sorted_front) - 1):
            distances[sorted_front[i]] += (values[sorted_front[i + 1]][m] - values[sorted_front[i - 1]][m]) / denom

    return distances



def fast_nondominated_sort(values):
    """NSGA-II's fast non-dominated sort"""
    S = [[] for _ in range(len(values))]
    front = [[]]
    n = [0 for _ in range(len(values))]
    rank = [-1 for _ in range(len(values))]
    
    for p in range(len(values)):
        S[p] = []
        n[p] = 0
        for q in range(len(values)):
            if dominates(values[p], values[q]):
                S[p].append(q)
            elif dominates(values[q], values[p]):
                n[p] += 1
        if n[p] == 0:
            rank[p] = 0
            front[0].append(p)
            
    i = 0
    while front[i]:
        nextFront = []
        for p in front[i]:
            for q in S[p]:
                n[q] = n[q] - 1
                if n[q] == 0:
                    rank[q] = i + 1
                    nextFront.append(q)
        i = i + 1
        front.append(nextFront)

    del front[len(front) - 1]
    
    # Initialize crowding distances as zeros
    crowding_distances = [0] * len(values)
    
    for front_solutions in front:
        current_front_distances = crowding_distance_assignment(front_solutions, values)
        for j, solution in enumerate(front_solutions):
            crowding_distances[solution] = current_front_distances[solution]
    
    return rank, crowding_distances


# Pipeline

In [32]:
def generate_random_pairs(list_of_arrays, n):
    # Generate all possible pairs
    random.seed(2020)
    all_pairs = list(itertools.combinations(list_of_arrays, 2))

    # Randomly select n pairs
    random_pairs = random.sample(all_pairs, n)

    return random_pairs

In [33]:
def generation(interaction, candidates, model, target, k, yloss_cache, crossover_p, mutation_p, budget):
    print(len(candidates))
    pairs = generate_random_pairs(candidates, len(candidates)//2)
    for first, second in pairs:
        budget -= 1
        first, second = crossover(first, second, crossover_p)
        first = mutate_array(interaction, first, mutation_p)
        second = mutate_array(interaction, second, mutation_p)
        first = remove_duplicates(first)
        second = remove_duplicates(second)
        candidates.append(first)
        candidates.append(second)
    print(len(candidates))
    losses = [compute_loss(interaction, arr, model, target, k, yloss_cache) for arr in candidates]
    budget -= len(candidates)
    print(losses)
    solved = False
    solved_list = []
    for i in range(len(losses)):
        if losses[i][0] == 0:
            solved = True
            solved_list.append(candidates[i])
    if solved:
        return solved_list, solved, budget
    ranks, crowding_distances = fast_nondominated_sort(losses)
    print(ranks)
    candidates_with_metrics = list(zip(candidates, ranks, crowding_distances))

    # Sort based on ranks (ascending) and then crowding distances (descending)
    candidates_with_metrics.sort(key=lambda x: (x[1], -x[2]))

    # Extract candidates after sorting
    sorted_candidates = [pair[0] for pair in candidates_with_metrics]

    # Extract the top third of candidates
    least_loss_arrays = sorted_candidates[:len(sorted_candidates)//2]

    return least_loss_arrays, solved, budget

In [34]:
def main(model, test_interaction, rank, sublists_info, top_k, crossover_p, mutation_p, budget):
    target = get_position_item(model, test_interaction, rank)
    new_gen = generate_random_sublists(test_interaction, sublists_info)
    budget -= 1
    solved = False
    yloss_cache = {}
    while solved is not True:
        new_gen, solved, budget = generation(test_interaction, new_gen, model, target, top_k, yloss_cache, crossover_p, mutation_p, budget)
        if budget == 0:
            break
    return new_gen, budget

In [35]:
# yloss_cache = {}
sublists_info ={
    8:5,
    9:5,
    10:5
}
# target = get_position_item(pooling_model, test_interaction, 1)
# first_gen = generate_random_sublists(test_interaction, sublists_info)
# generation(test_interaction, first_gen, lstm_model, target, 10, yloss_cache, 0.3, 0.2)
main(pooling_model, test_interaction, 1, sublists_info, 10, 0.3, 0.2, 1000)

15
29
[[0.15775024890899658, 3, 1.0352857112884521], [0.09447741508483887, 4, 3.743652820587158], [0, 3, 2.246750593185425], [0.13993871212005615, 3, 2.6697992086410522], [0, 5, 9.13612186908722], [0.06763672828674316, 3, 6.433834075927734], [0.03534376621246338, 4, 6.556341886520386], [0.11601829528808594, 3, 4.803943395614624], [0.03865993022918701, 3, 3.9157055616378784], [0.13430464267730713, 2, 4.7797956466674805], [0.020376086235046387, 4, 10.445356130599976], [0.13138890266418457, 4, 7.883714199066162], [0.5975372791290283, 1, 1.0352857112884521], [0.6852766275405884, 1, 1.0352857112884521], [0.20534729957580566, 4, 16.949491500854492], [0.11601829528808594, 3, 5.509027481079102], [0.07860791683197021, 3, 10.648540377616882], [0.09447741508483887, 4, 13.043522000312805], [0.07860791683197021, 3, 10.648540377616882], [0.03865993022918701, 3, 8.407523155212402], [0.020376086235046387, 4, 15.820550203323364], [0.020376086235046387, 4, 15.820550203323364], [0.2229156494140625, 5, 34

([array([114, 190, 177, 125, 124, 191, 200,  59], dtype=int32),
  array([186, 200, 124, 186, 186, 114, 177, 125], dtype=int32),
  array([186, 200, 124, 191, 114, 177, 125], dtype=int32),
  array([186, 200, 124, 114, 177, 125], dtype=int32)],
 963)

# Brute Force Search
Find hard cases

In [36]:
import numpy as np
import itertools

def subsets_of_array(arr, k):
    if not isinstance(arr, np.ndarray):
        raise ValueError("Input must be a numpy array")

    n = len(arr)
    subsets = []

    for size in range(n-1, n-k-1, -1):
        combinations = itertools.combinations(arr, size)
        for combo in combinations:
            subsets.append(np.array(combo))

    return subsets

In [37]:
def compute_loss_brute(new_cf, model, target_item, top_k, yloss_cache):
    cache_key = frozenset(new_cf)
    if cache_key in yloss_cache:
        yloss = yloss_cache[cache_key]
    else:
        new_prediction = model.predict(new_cf)
        new_prediction[new_cf] = -StaticVars.FLOAT_MAX
        new_rk_data = st.rankdata(-new_prediction, method='ordinal')

        top_k_index = np.where(new_rk_data == top_k)[0][0]
        yloss = compute_yloss(new_prediction[target_item], new_prediction[top_k_index])
        yloss_cache[cache_key] = yloss
    return yloss

In [38]:
def brute(candidates, model, target, top_k, yloss_cache):
    losses = [compute_loss_brute(arr, model, target, top_k, yloss_cache) for arr in candidates]
    solved = False
    for i in range(len(losses)):
        if losses[i] == 0:
            solved = True
    return solved

In [39]:
def brute_main(model, test_interaction, rank, top_k):
    target = get_position_item(model, test_interaction, rank)
    new_gen = subsets_of_array(test_interaction, 3)
    solved = False
    yloss_cache = {}
    solved = brute(new_gen, model, target, top_k, yloss_cache)
    return solved

In [40]:
brute_main(pooling_model, test_interaction, 1, 10)

True

In [67]:
test.sequences[13130]

array([   0,    0,    0,    0, 1552,  168,   39,  521,  272,   56,  281,
        277,  454, 1236, 1099,   72, 2708,   85,  393,  626], dtype=int32)

In [71]:
final_list = []
for i in range(1000):
    test_interaction = test.sequences[i].copy()
    est_interaction = test_interaction[test_interaction != 0]
    test_interaction.sort()
    for j in range(1, 11):
        solved = brute_main(pooling_model, test_interaction, j, 10)
        if not solved:
            print(i, j)
            final_list.append((i, j))

with open('final_list.txt', 'w') as file:
    for item in final_list:
        file.write(f"{item[0]}, {item[1]}\n")

1 1
1 2
4 1
4 2
4 3
4 4
4 5
5 1
5 2
5 3
7 1
10 1
10 2
11 1
11 2
11 3
12 2
13 5
14 1
14 2
14 3
16 1
16 4
17 1
17 2
19 2
19 3
20 1
21 1
23 1
23 2
24 1
24 3
24 4
24 6
24 8
24 9
26 1
26 2
26 3
26 4
26 5
27 1
28 1
31 1
32 1
32 2
35 8
36 2
37 1
37 2
38 1
40 1
40 2
40 3
41 1
43 1
43 2
44 1
45 2
46 1
48 1
48 2
48 3
49 2
51 1
52 3
54 1
54 2
58 1
58 2
58 3
59 1
60 2
61 1
61 2
62 1
64 1
65 1
66 1
68 1
69 3
70 1
71 1
74 1
74 2
74 3
74 4
75 1
75 2
76 1
77 1
80 1
80 2
80 5
82 1
83 1
84 1
84 2
84 3
84 4
84 5
84 6
85 2
86 1
86 5
89 1
89 2
90 1
90 2
91 1
91 2
92 1
92 3
93 1
96 1
96 3
96 5
99 1
99 2
99 5
100 1
102 1
102 3
102 4
102 5
103 1
103 2
104 1
105 1
105 2
106 1
106 4
106 5
107 1
107 2
112 1
113 1
113 2
114 1
115 2
116 2
117 2
117 3
118 1
118 2
118 5
119 3
121 1
121 2
124 1
124 2
124 3
125 1
126 1
126 3
126 5
128 1
129 1
130 1
131 1
131 2
131 3
133 1
133 2
133 4
136 1
136 2
136 4
137 1
137 2
137 3
138 2
140 1
141 1
142 1
142 3
142 4
143 1
143 2
144 1
145 1
145 2
145 4
145 5
147 1
147 2
149 1
149 